In [1]:
import os
import time
import json

from utils.file_utils import *
from datasets import list_datasets
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from IPython import get_ipython
import pandas as pd
from tqdm import tqdm
from Sai_Test.MergeModel import *
from Sai_Test.ClassifierModel import *
import matplotlib.pyplot as plt
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification
from datasets import load_dataset
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', 2)

In [2]:
!ls

argmax.py			     MergeModel_20_MSE_eqcost.pt
BART_classifier_final_MSE_eqcost.pt  MergeModel_20_MSE.pt
BART_classifier_final_MSE.pt	     MergeModel_30_MSE_eqcost.pt
BART_training.ipynb		     MergeModel_30_MSE.pt
Classification_XLNet.ipynb	     MergeModel_40_MSE_eqcost.pt
classifier_final.pt		     MergeModel_40_MSE.pt
ClassifierModel_o.py		     MergeModel_49_MSE_eqcost.pt
classifier_model.pt		     MergeModel_49_MSE.pt
ClassifierModel.py		     MergeModel_Final.ipynb
classifier.pt			     MergeModel_Train.ipynb
dataset				     MergeModel_Train_MSE.ipynb
differentiable_embedding.py	     MergeModel_Train_MSE_new.ipynb
Embedding_differentiable.ipynb	     models
experiment			     __pycache__
gumbel_softmax.py		     README.md
in_domain_dev.tsv		     Sai_Test
in_domain_train.tsv		     Sentiment_Classifier.ipynb
kl_div.py			     softargmax.py
LSTM_Tests.ipynb		     Untitled.ipynb
MergeModel_10_MSE_eqcost.pt	     utils
MergeModel_10_MSE.pt


In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
vocab_size = tokenizer.vocab_size

In [4]:
sent_model = ClassifierModel(vocab_size, 64, 2, 2, 512)
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [5]:
merge_model = MergeModel(summary_model, sent_model)

In [6]:
merge_model.load_state_dict(torch.load('BART_classifier_final_MSE_eqcost.pt'))
merge_model = merge_model.cuda()

CLF

In [7]:
dataset = load_dataset("imdb")

Reusing dataset imdb (/tmp/xdg-cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [8]:
train_batch = dataset['train']['text']
train_labels = torch.tensor(dataset['train']['label'])
val_batch = dataset['test']['text']
val_labels = torch.tensor(dataset['test']['label'])

In [9]:
seq_length = 150
input_encoding = tokenizer(train_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)
val_encoding = tokenizer(val_batch, return_tensors='pt', padding=True, truncation = True, max_length=seq_length)

In [10]:
input_ids = input_encoding['input_ids']
input_mask = input_encoding['attention_mask']

In [11]:
val_ids = val_encoding['input_ids']
val_mask = val_encoding['attention_mask']

SUMMARY

In [12]:
df = pd.read_csv("dataset/test.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 11,490



,article,highlights,id
6158,A woman who already has four children is set t...,"Nadine Crooks, 33, has four children aged 18 y...",7b1bd79278c2ced9eb271a24b8911dd7147df6ff
8265,These incredible photographs appear to captur...,Cloud formation before the red dusk sky resemb...,b07c92c4891209b7e43ae2bbb851905b6169ad2e
11389,Prince Harry will go on tough bush patrols and...,Details of Prince Harry's Australian deploymen...,fd3a7a63c35a929ab9ab228def89fa73591892d0
2386,In cities that are as well-known for their bur...,Hotels around the world are now hiring top des...,1f25f86d975211c64655e6e8802651fc6e66a83b
10527,Not a day goes by that 14-year-old Jonathan Pi...,Jonathan Pitre has deep blistering wounds all ...,e80a5379065c0cc87670ef2ce3f3f378904492a3
11291,Model and actress Sulinh Lafontaine is not a d...,Sulinh Lafontaine presented herself as a dared...,fab2ad3269eef684b7da01efe151b4b1ce6a2596
6297,The NSW State Emergency Service (SES) are warn...,NSW SES warns scammers are phoning people clai...,7ead02c636ed9db09cdb973b943a61587ef627e7
3686,Staff attorneys at the U.S. Justice Department...,Attorneys at the U.S. Justice Department may r...,3f2f668a7dce47fe7dd7ff57a1e796245a7cb754
5275,"Lesley Conman's daughter Abbie, 12, was sent s...",Young girls were sent sexualised Facebook mess...,64d847a2f06d277f895e41e5f751c6a8fb76f136
2336,Canadian pop sensation Justin Bieber is will c...,Justin Bieber is a good friend of Floyd Maywea...,1dc22202efb4e51fb9487ccf82524ab9eb2de7b5


In [13]:
train_sentence = df.article.values
train_target = df.highlights.values


num_data_points = 500
train_sentence = list(train_sentence)[:num_data_points]
train_target = list(train_target)[:num_data_points]

In [14]:
article_encoding = tokenizer(train_sentence, return_tensors='pt', padding=True, truncation = True, max_length=500)
summary_encoding = tokenizer(train_target, return_tensors='pt', padding=True,truncation = True, max_length=100)

In [15]:
article_input_ids = article_encoding['input_ids']
article_attention_mask = article_encoding['attention_mask']

In [16]:
summary_input_ids = summary_encoding['input_ids']
summary_attention_mask = summary_encoding['attention_mask']

In [17]:
batch_size = 16

train_data = TensorDataset(input_ids, input_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader_clf = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [18]:
batch_size = 4
train_data = TensorDataset(article_input_ids, article_attention_mask,\
                           summary_input_ids, summary_attention_mask)

In [19]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [20]:
from nltk.translate.bleu_score import sentence_bleu

In [21]:
def get_bleu(reference, candidate):
    score1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))    
    return score1

In [22]:
c1,c2,c3 = next(iter(train_dataloader_clf)) 
c1 = c1.cuda() 
c2 = c2.cuda() 
c3 = c3.cuda() 

In [23]:
b1,b2,b3,b4 = next(iter(train_dataloader)) 
b1 = b1.cuda() 
b2 = b2.cuda() 
b3 = b3.cuda() 
b4 = b4.cuda()

In [24]:
tokenizer.decode(b1[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

'Catania, Sicily (CNN)We are at the beginning of a massive and mounting crisis with no solution in sight. Perhaps that\'s incorrect. The migrant crisis that has suddenly drawn hundreds of journalists to Sicily has been brewing for years, but in the past 10 days, with as many as 1,600 deaths in the Mediterranean, suddenly minds are focused -- for now. Almost exactly four years ago, in Libya, I caught, perhaps, a glimpse of what was to come. It was late at night in the besieged city of Misrata. Hundreds of African migrants were caught between the Libyan civil war (back then some optimistically called it a "revolution") and the deep blue sea. They had come to Misrata from Ghana, Nigeria and elsewhere, hoping to board rickety boats to cross the sea to Europe. They had been pinned down under sporadic shelling from government forces, but weren\'t welcome by the rebels who controlled the city. They appealed to us to help them escape. We could do nothing, but they may have eventually found the

In [25]:
tokenizer.decode(b3[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

'Hundreds of desperate migrants have died attempting to cross the Mediterranean in recent says.\nAnd Italians are alarmed that this year as many as a million migrants could arrive in Europe.'

In [26]:
summary_out,*sentiments = merge_model(b1, b2, b3, b4)

In [27]:
tokenizer.decode(summary_out.logits.argmax(dim = -1)[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

'Hundreds of journalists African have been in to cross the Mediterranean. recent days Italian\nItaly now are alarmed at this year alone many as a million migrants could arrive in Europe.\nTheItaly Italian the the the Italian the the the theItaly Italy Italy the the the theItaly the the the the the the the the the the the the the the the the the the the the Mediterranean thely the the the the thely the thely the'

In [28]:
merge_model.load_state_dict(torch.load('BART_classifier_final_MSE_eqcost.pt'))
merge_model = merge_model.cuda()

In [29]:
sent_model.load_state_dict(torch.load('Sai_Test/classifier_model.pt'))
sent_model = sent_model.cuda()
sent_model = sent_model.eval()

In [30]:
temp = []
ss = []

for b1, b2, b3, b4 in train_dataloader:
    b1 = b1.cuda() 
    b2 = b2.cuda() 
    b3 = b3.cuda() 
    b4 = b4.cuda()
    summary_out,*sentiments = merge_model(b1, b2, b3, b4)
    ss.append(torch.abs(sent_model(summary_out.logits.argmax(dim = -1)) - sent_model(b1)/6).mean().item())
    
    for i in range(b3.shape[0]):
        summary_gt = tokenizer.decode(b3[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summary_pred = tokenizer.decode(summary_out.logits.argmax(dim = -1)[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        temp.append(get_bleu(summary_pred, summary_gt))
        
print(np.array(temp).mean(), np.array(ss).mean())

/datasets/home/11/811/adharask/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/datasets/home/11/811/adharask/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/datasets/home/11/811/adharask/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it c

0.1586451536859683 0.21263009071350097


In [31]:
merge_model.load_state_dict(torch.load('BART_classifier_final_MSE.pt'))
merge_model = merge_model.cuda()

In [32]:
temp = []
ss = []

for b1, b2, b3, b4 in train_dataloader:
    b1 = b1.cuda() 
    b2 = b2.cuda() 
    b3 = b3.cuda() 
    b4 = b4.cuda()
    summary_out,*sentiments = merge_model(b1, b2, b3, b4)
    ss.append(torch.abs(sent_model(summary_out.logits.argmax(dim = -1)) - sent_model(b1)/6).mean().item())
    
    for i in range(b3.shape[0]):
        summary_gt = tokenizer.decode(b3[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summary_pred = tokenizer.decode(summary_out.logits.argmax(dim = -1)[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        temp.append(get_bleu(summary_pred, summary_gt))
        
print(np.array(temp).mean(), np.array(ss).mean())

0.15852145927668804 0.0426627171933651
